In [18]:
import tiktoken
import torch

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4")
print(enc)

<Encoding 'cl100k_base'>


In [17]:
en =enc.encode("ishaque loves wet pussy ")
print(en)
de = enc.decode(en)
print(de)

[819, 20234, 16180, 14739, 16585, 220]
ishaque loves wet pussy 


In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("charachetr length"  ,len(text))

charachetr length 5370734


In [5]:
print(text[:1000])

name: A Channel plot summary:  Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
name: A Channel Ova  plot summary:  Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
name: A Channel Special  plot summary:  Toeru takes a test so she can enter the same high school as Run, the girl she likes. She passes, but when she goes   to tell Run, she finds her hugging a girl she’s never seen before.
name: A Day Before Us plot summary:  A Day Before Us is an animated Korean drama of a special and sweet love story of the following four main characters; Yeo Reum, Ha Eun, Kim Wook, and Yeon Woo.
name: A Disguised Princess plot summary:  "Ken always disguised herself as a man. She was totally confused about why she got entangled with that 

In [9]:
chars = sorted(list(set(text)))
print(len(chars))

283


In [19]:
data = torch.tensor(enc.encode(text))

TypeError: argument 'tokens': 'Tensor' object cannot be converted to 'Sequence'